1. train RoSTER model on out-of-domain weak data
    * train on weak labels generated onto Wikigold or OntoNotes datasets via knowledge bases --> full CONTROSTER: double backbone approach, i.e., train RoSTER first on either Wikigold dataset (13041 lines of training data, 3 overlapping entity categories with COVIDNews out of 4 total entity categories) or OntoNotes5.0 dataset (59924 lines of training data, 5 overlapping entity categories with COVIDNews out of 18 total entity categories) 
    * data from meng2021 (RoSTER)
    * "Even though the Wikigold dataset had only three overlapping entity categories with COVIDNEWS-NER and OntoNotes5.0, the performance still improved in part due to the overlapping categories being the prominent ones in COVIDNEWS-NER (eg. organisation, person, location, date)" (ficek2022, p.5)
    
2. repeat RoSTER training on in-domain weak data

3. fine-tune on strong data with only noise-robust loss


### Step 1: Run RoSTER on Wikigold

#### Step 1 in meng2021:
```python 
export CUDA_DEVICE_ORDER=PCI_BUS_ID
export CUDA_VISIBLE_DEVICES=0

CORPUS=wikigold
SEED=30
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_$CORPUS
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir data/$CORPUS --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 120 \
    --train_batch_size 32 --gradient_accumulation_steps 1 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 5 --ensemble_train_epochs 10 --self_train_epochs 5 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion=0.1 --seed $SEED \
    --q 0.7 --tau 0.7 --num_models 5 \
    --do_train --do_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
```

#### Step 1 in ficek2022:
```python
sh = """
CORPUS=wiki_w1
SEED=30
TEMP_DIR=tmp_{CORPUS}_SEED
OUT_DIR=out_TEMP_DIR
mkdir -p OUT_DIR --temp_dir SEED \
    --q 0.7 --tau 0.7 --num_models 5 \
    --do_train --do_eval --eval_on "test" --save_model | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
    file.write(sh)
!bash script.sh
```

### Step 2: Continue training on in-domain weak data


```python
sh = """
CORPUS=covidnews
SEED=30
TEMP_DIR=tmp_{CORPUS}_SEED
OUT_DIR=out_TEMP_DIR
mkdir -p CORPUS --output_dir TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 2 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 50 --ensemble_train_epochs 10 --self_train_epochs 5 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion 0.1 --seed SEED \ --num_models 1 --tau 0.0 --q 0.0 --backbone_dir_s1 out_wiki_w_bio_w_2000 --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee OUT_DIR/train_log.txt 
"""
with open('script.sh', 'w') as file:
    file.write(sh)
!bash script.sh
```